In [53]:
import pandas as pd
from sklearn.impute import SimpleImputer

data = pd.read_csv('./train.csv')
data.dropna(axis=0, subset=['SalePrice'], inplace=True)
train_y = data.SalePrice
train_X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

data = pd.read_csv('./test.csv')
test_X = data.select_dtypes(exclude=['object'])

train_X=train_X.to_numpy()
test_X=test_X.to_numpy()

my_imputer = SimpleImputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

In [54]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(train_X, train_y, verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [55]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=10)
my_model.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [56]:
my_model.score(train_X, train_y)

0.9999999999475051

In [57]:
import numpy as np

temp=my_model.predict(test_X)

In [58]:
pd.DataFrame(temp, index=data["Id"], columns=["SalePrice"]).to_csv("output.csv")

In [63]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [65]:
n_folds = 5

def rmsle(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse = np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [134]:
for est in [500, 1000, 2000, 3000, 5000]:
    for lrn in [0.05, 0.08, 0.1, 0.3, 0.5]:
        for dpt in range(4, 10):
            model_xgb = xgb.XGBRegressor(n_estimators=est, learning_rate=lrn, max_depth=dpt)

            model_xgb.fit(train_X, train_y)
            xgb_train_pred = model_xgb.predict(train_X)
            xgb_pred = model_xgb.predict(test_X)
            print(rmsle(test_y, xgb_pred))

            if rmsle(test_y, xgb_pred) < Bestscore:
                Bestscore=rmsle(test_y, xgb_pred)
                feature=model_xgb.get_params


25008.56489699013
25366.973882098726
25464.588999559855
25106.51393181682
25591.586672904377
25631.782551846754
25091.691303928295
25790.12609184402
25535.69512191831
25638.640981483433
25906.57153885655
26547.696231072736
25876.188357057217
25535.952653410408
25614.096453584294
25780.117779010416
26155.648429117104
26397.291776587517
27176.252201462787
26858.88111029756
26400.93781363299
27774.348350211236
28091.668783504094
28492.547287511436
27536.598117100235
29307.984014728376
31296.19885362624
31749.802789459285
31956.838356733802
32935.67187604353
25032.863634302958
25373.95160546144
25397.990005892618
25106.061357594914
25585.890689316966
25632.055020537267
25113.55827009802
25817.48129308794
25540.654998966
25642.140329537186
25905.642949186644
26547.322849268607
26043.039392940645
25557.52765149538
25611.09483971324
25779.086779319365
26155.361896758237
26397.111900829666
27170.35369625519
26858.539032502726
26400.96170608175
27774.34654884919
28091.66794582481
28492.51994538

In [147]:

model_xgb = xgb.XGBRegressor(n_estimators=500, learning_rate=0.06, max_depth=4)

model_xgb.fit(train_X, train_y)
xgb_train_pred = model_xgb.predict(train_X)
xgb_pred = model_xgb.predict(test_X)
print(rmsle(test_y, xgb_pred))

if rmsle(test_y, xgb_pred) < Bestscore:
    Bestscore=rmsle(test_y, xgb_pred)
    feature=model_xgb.get_params


24683.746383420857


In [138]:
print(Bestscore, feature)

25008.56489699013 <bound method XGBModel.get_params of XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)>


In [148]:
temp=model_xgb.predict(test_X)
pd.DataFrame(temp, index=data["Id"], columns=["SalePrice"]).to_csv("output.csv")

In [274]:
import catboost as cb
from catboost import Pool
from sklearn.model_selection import train_test_split

grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}

model_cb = cb.CatBoostRegressor(iterations=700, learning_rate=0.1, depth=6, l2_leaf_reg=0.2)
model_cb.fit(train_X, train_y)
cb_pred = model_cb.predict(test_X)
print(rmsle(test_y, cb_pred))
model_cb

0:	learn: 74001.3550935	total: 3.39ms	remaining: 2.37s
1:	learn: 68988.3612663	total: 6.09ms	remaining: 2.12s
2:	learn: 64154.3822325	total: 8.17ms	remaining: 1.9s
3:	learn: 60257.3280064	total: 10.5ms	remaining: 1.83s
4:	learn: 56969.1805620	total: 12.7ms	remaining: 1.76s
5:	learn: 53855.3748694	total: 15.3ms	remaining: 1.77s
6:	learn: 50862.7650124	total: 17.8ms	remaining: 1.76s
7:	learn: 48311.6935769	total: 20.3ms	remaining: 1.76s
8:	learn: 45953.7483337	total: 22.8ms	remaining: 1.75s
9:	learn: 43611.8621292	total: 24.9ms	remaining: 1.72s
10:	learn: 41520.4106744	total: 27ms	remaining: 1.69s
11:	learn: 39683.4446331	total: 29.4ms	remaining: 1.68s
12:	learn: 38140.9795525	total: 31.7ms	remaining: 1.68s
13:	learn: 36679.9951065	total: 34ms	remaining: 1.67s
14:	learn: 35407.5556329	total: 36.3ms	remaining: 1.66s
15:	learn: 34180.7965606	total: 38.9ms	remaining: 1.66s
16:	learn: 32986.0688175	total: 40.9ms	remaining: 1.64s
17:	learn: 31879.6720134	total: 42.9ms	remaining: 1.63s
18:	lea

In [258]:
temp = model_cb.predict(test_X)
pd.DataFrame(temp, index=data["Id"], columns=["SalePrice"]).to_csv("output.csv")